<h4>Librerie Necessarie:</h4>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

<h4>Lettura del DataFrame:</h4>

In [ ]:
df = pd.read_csv("../data/star_classification.csv")

In [ ]:
df.head()

In [ ]:
df.info()

<h4>Come possiamo notare, non abbiamo valori nulli. Ottimo per poterci alleggerire la fase di pulizia dei dati</h4>

In [ ]:
df.describe()

<h3>1. PULIZIA DEI DATI</h3>

<h4>Rinominazione delle colonne: u, g, r, i, z, al fine di garantire maggiore chiarezza dei dati</h4>

In [ ]:
df.rename(columns={
    "u": "ultraviolet",
    "g": "green",
    "r": "red",
    "i": "near_infrared",
    "z": "infrared"
}, inplace=True) # con il valore andiamo ad indicare di andare a modificare il dataframe originale, senza creare una copia

<p>Spiegazione colonne DataFrame: </p>
<ul>
    <li>Alpha e delta: posizione dell'oggetto nel cielo</li>
    <li>U, g, r, i, z: spettro di luce che emana l'oggetto</li>
    <li>Class: classificazione dell'oggetto</li>
    <li>Redshift: misura quanto la luce di un oggetto è spostato verso il rosso rispetto a come viene emessa</li>
    <li>Plate: tipo di fibra usato nell'obbiettivo</li>
    <li>MJD: data in cui è stata scattata la foto, basandosi sul calendario Giuliano modificato</li>
</ul>

<h4>Eliminazione colonne superflue per l'esplorazione dei dati</h4>

In [ ]:
uselessColumns = ['run_ID', 'rerun_ID', 'cam_col', 'fiber_ID', 'field_ID']
df = df.drop(uselessColumns, axis=1)

print(f"Sono state cancellate le seguenti colonne: {uselessColumns}")

<h4>Eliminazione righe dove contengono valori sentinella. Essi sbilanciano di molto analisi future, meglio rimuoverle</h4>

In [ ]:
df = df[
    (df["ultraviolet"] > -1000) &
    (df["green"] > -1000) &
    (df["infrared"] > -1000)
]

In [ ]:
df.describe()

<h3>2. Analisi Esplorativa Dei Dati</h3>

<h4>a. Contiamo le ricorrenze della colonna target "class"</h4>

In [ ]:
sns.countplot(data=df, x="class", hue="class")
plt.title("Tipologia di oggetto cosmico",fontsize=10)
plt.show()

In [ ]:
data = df["class"].value_counts()
labels = ['GALAXY', 'STAR', 'QSO']
# define Seaborn color palette to use
colors = sns.color_palette('pastel')

# plotting data on chart
plt.pie(data, labels=labels, colors=colors, autopct='%.0f%%')
plt.show()
data

<h5>Possiamo notare come la classe prioritaria "GALAXY" sia circa il 60% dei dati complessivi, rispetto alle altre due minoritarie molto simili, "START" e "QSO", che si attestano intorno al 20%. Il DataSet non presenta un forte sbilanciamento.</h5>

<h4>b. Stampiamo la HEATMAP per verificare la correlazione delle variabili</h4>

<h5>Applichiamo una funzione encoder sulle colonne non numeriche</h5>

In [ ]:
def encode_class(value):
    if value == "GALAXY":
        return 0
    elif value == "STAR":
        return 1
    else:
        return 2
    
df_encoded = df.copy()
df_encoded["class"] = df_encoded["class"].apply(encode_class)

In [ ]:
f,ax = plt.subplots(figsize=(12,8))
sns.heatmap(df_encoded.corr(), cmap="coolwarm", annot=True, linewidths=1, fmt= '.2f',ax=ax)
plt.show()

<h4>c. Relazione tra bande fotometrice tramite scatterplot</h4>

In [ ]:
# sns.pairplot(
#     df[["ultraviolet", "green", "red", "near_infrared", "infrared", "class"]],
#     hue="class",
#     diag_kind="hist",
#     plot_kws={"alpha": 0.5, "s": 20}
# )

# plt.show()


<h5>Si può notare come, in alcuni scatterplot, non si riesca a differenziare bene la differenza di classe, sovrapponendosi.</h5>

<h5>Quelli che sembrano dividere meglio sono: </h5>

<ul>
    <li>Red and Near_infrared</li>
    <li>ultraviolet and infrared, differenza tra STAR e GALAXY ma non buona per QSO</li>
    <li>Green and Near_infrared, uguale per i QSO</li>
</ul>

<h4>d. Istogramma bande magnitudinali</h4>

In [ ]:
data = df[["ultraviolet", "green", "red", "near_infrared", "infrared"]].copy()



for d in data.columns:
    plt.figure(figsize=(5, 2))
    sns.histplot(x=data[d], kde=True, bins=50)
    plt.title(f"Istogramma banda magnitudinale: {d}")
    plt.show()

<h4>d. Mostriamo come, tramite scatterplot, la luminosità degli oggetti stellari cambia in relazione alla loro distanza</h4>

In [ ]:
sns.scatterplot(data=df, x="redshift", y="red", alpha=0.5)
plt.xlabel("Redshift")
plt.ylabel("Magnitudine in banda r")
plt.title("Magnitudine vs Redshift")
plt.show()


<h4>e. Classificazione oggetti stellari tramite le loro bande</h4>
<h5>Ogni stella, viene categorizzata in valori tramite</h5>

In [ ]:
stars = df[df["class"] == "STAR"].copy()

In [ ]:
stars["u_g"] = stars["ultraviolet"] - stars["green"]
stars["g_r"] = stars["green"] - stars["red"]
stars["r_i"] = stars["red"] - stars["near_infrared"]
stars["i_z"] = stars["near_infrared"] - stars["infrared"]

In [ ]:
def spectral_type_from_gr(g_r):
    if g_r < -0.2:
        return "O/B"
    elif g_r < 0.0:
        return "A"
    elif g_r < 0.3:
        return "F"
    elif g_r < 0.6:
        return "G"
    elif g_r < 1.0:
        return "K"
    else:
        return "M"
    
stars["spectral_type"] = stars["g_r"].apply(spectral_type_from_gr)

In [ ]:
stars[["obj_ID", "alpha", "delta", "ultraviolet", "green", "red", "g_r", "spectral_type"]].head()

In [ ]:
sns.histplot(x=stars["spectral_type"])
plt.title("Tipi Spettrali")
plt.show()

<h5>La maggior parte di stelle all'interno del nostro DATASET hanno classe spettrale G, simili al nostro sole!</h5>
<h5>La temperatura si aggira circa tra i 5.300 k e i 6.000 k</h5>

In [ ]:
plt.figure(figsize=(10,8))

for t in ["O/B", "A", "F", "G", "K", "M"]:
    sub = stars[stars["spectral_type"] == t]
    plt.scatter(sub["g_r"], sub["r_i"], s=5, alpha=0.5, label=t)

plt.xlabel("g − r")
plt.ylabel("r − i")
plt.title("Stellar locus by spectral type")
plt.legend(title="Spectral type")
plt.show()

<em>log<sub>10</sub>(T<sub>eff</sub>) = 3.877 − 0.26(g−r)</em>

In [ ]:
stars = stars[stars["g_r"].between(-0.3, 1.0)]

In [ ]:
stars["Teff_K"] = 10 ** (3.877 - 0.26 * stars["g_r"])

In [ ]:
stars["Teff_K"].describe()

In [ ]:
plt.figure(figsize=(7,4))
plt.hist(stars["Teff_K"], bins=60)
plt.xlabel("Teff [K]")
plt.ylabel("Numero di stelle")
plt.title("Distribuzione della temperatura efficace")
plt.show()

In [ ]:
stars["r_i"] = stars["red"] - stars["near_infrared"]

plt.figure(figsize=(7,6))
sc = plt.scatter(
    stars["g_r"],
    stars["r_i"],
    c=stars["Teff_K"],
    s=6,
    alpha=0.5
)
plt.xlabel("g − r")
plt.ylabel("r − i")
plt.title("Locus stellare colorato per Teff")
plt.colorbar(sc, label="Teff [K]")
plt.show()

<h4>Da mdefinirei</h4>


In [ ]:
plt.colorbar(sc, label="Teff [K]")